In [1]:
#pip install requests
#pip install bs4
#pip install spacy
#pip install pymongo
#pip install spacy download https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0.tar.gz
#pip instal gensim
#pip install word2number
#pip install inflect
#pip install PyPDF2

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import date
from datetime import datetime
import spacy
import sys
from pymongo import MongoClient
import re
from word2number import w2n
import inflect

import nltk
from gensim import corpora
import random
from nltk.corpus import wordnet as wn  
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English

mainUrl = "https://kyivindependent.com/wp-json/wp/v2/posts/?orderby=date&per_page=100&page="

payload={}
headers = {
  'authority': 'kyivindependent.com',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'accept-language': 'it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7,fr;q=0.6',
  'cache-control': 'no-cache',
  'cookie': '_hjSessionUser_2741431=eyJpZCI6IjhmODQ0YjgyLWRmMzktNTg4OC1hMzAxLTkwZTUwNzlhYzQ5NyIsImNyZWF0ZWQiOjE2NDU3Mjk3MTcwNzMsImV4aXN0aW5nIjp0cnVlfQ==; cf_clearance=0B7qGXQrTfbNX6JsyaQag6ZGakcuBPEV8JgBYNQCvyA-1646492515-0-150; cookie_notice_accepted=true; _gid=GA1.2.163750293.1648756295; _hjSession_2741431=eyJpZCI6IjQ2ZDQwNTNkLTgxZDQtNDE0Ny04ZTVhLTkwZjM0MDFiMWFlNyIsImNyZWF0ZWQiOjE2NDg3NTYyOTUxNTIsImluU2FtcGxlIjpmYWxzZX0=; _hjAbsoluteSessionInProgress=0; _hjIncludedInSessionSample=0; _ga_4WHBG5CRKW=GS1.1.1648756294.17.1.1648759737.0; _ga=GA1.2.1064471579.1645729717; mf_43294eee-4fef-49b5-83b6-fe216cdec496=|.12563616317.1648759737756|1648083465574||0|||0|0|18.86952',
  'pragma': 'no-cache',
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'none',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'
}

nlp = spacy.load("en_core_web_sm")
inflectEngine = inflect.engine()
spacy.load('en_core_web_sm')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
nltk.download('omw-1.4')

end_parsing_string = "The page number requested is larger than the number of pages available."

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dude\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dude\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dude\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def new_article_model(title, date, main, tag, custom_tag, deaths):
    return {
        'title': title,
        'date': date,
        'main': main,
        'tag': tag,
        'custom_tag': custom_tag,
        'deaths': deaths
    }

def get_text_tokenized(text):
    token_object = prepare_text_for_lda(text)
    tokens = choose_best_lemma(token_object)
    return remove_tag_duplicates(tokens)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    tokens2 = [get_lemma2(token) for token in tokens]
    return {
        "first_lemma": tokens,
        "second_lemma": tokens2
    }

def tokenize(text):
    lda_tokens = []
    tokens = nlp(text)

    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    return word if lemma is None else lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def best_lemma(lemma1, lemma2):
    return lemma2 if len(lemma1) > len(lemma2) else lemma1

def choose_best_lemma(token_object):
    if token_object['first_lemma'] is None and token_object['second_lemma'] is None:
        return []
    if token_object['first_lemma'] is None:
        return token_object['second_lemma']
    if token_object['second_lemma'] is None:
        return token_object['first_lemma']
    return best_lemma(token_object['first_lemma'], token_object['second_lemma'])

def remove_tag_duplicates(tags):
    tags_cleaned = []
    for tag in tags:
        if tag not in tags_cleaned:
            tags_cleaned.append(tag)
    return tags_cleaned

def convert_numbers_in_text(text):
    number_word_list = numbers_word_list(inflect_engine)
    for number in number_word_list:
        if number in text:
            text = text.replace(number, str(w2n.word_to_num(number)))
    return text

def numbers_word_list(inflect_engine):
    numbers_list = []
    for i in range(100):
        number_to_add = inflect_engine.number_to_words(i)
        numbers_list.append(number_to_add)
        if '-' in number_to_add:
            numbers_list.append(number_to_add.replace('-', ''))
    return numbers_list

def search_death(article_body):
    values = []
    for synonym in synonims_deaths:
        value = search_death_regex(synonym, article_body)

        if len(value) > 0:
            single_value = []
            for s_value in value:
                if s_value is not None and len(s_value) > 0:
                    for any_val in s_value:
                        if len(any_val) > 0:
                            single_value.append(any_val)

            for val in single_value:
                if val not in values:
                    values.append(val)
    values_int = [int(v.replace(',', '')) for v in values]
    return values_int

def search_death_regex(synonym, text):
    expression = r"(?i)(?:\b" + synonym + r"\D{0,20})([0-9][0-9,]*)[^.,]|([0-9][0-9,]*)[^.,](?:\D{0,20}" + synonym + ")"
    values = re.findall(expression, text)
    return values

def get_content_text(article_json):
    soup_text = BeautifulSoup(article_json['content']['rendered'], 'html5lib')
    return soup_text.text.replace('\n', '')

def get_death_value(body_article):
    article_body_converted = convert_numbers_in_text(body_article)
    death_value = search_death(article_body_converted)
    return death_value

def date_str_to_datetime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S')

def get_tag_from_slug(article_json):
    slug = article_json['slug']
    slug_update = slug.replace('-', ' ')
    tags = get_text_tokenized(slug_update)
    return tags

def convert_json_page_to_articles_list(page_json, articles_list):
    for article_json in page_json:
        body_article = get_content_text(article_json)
        custom_tags = get_text_tokenized(body_article)
        death_value = get_death_value(body_article)
        date = date_str_to_datetime(article_json['date'])
        title = article_json['title']['rendered']
        tags = get_tag_from_slug(article_json)
        new_article = new_article_model(title, date, body_article, tags, custom_tags, death_value)
        articles_list.append(new_article)
    return articles_list

def remove_duplicates(article_list):
    new_list = []
    article_titles = set()
    duplicates_count = 0
    for article in article_list:
        if article['title'] not in article_titles:
            article_titles.add(article['title'])
            new_list.append(article)
        else:
            duplicates_count += 1
    return new_list

def connect_mongo(mongo_url, mongo_db):
    client = MongoClient(mongo_url)
    mongo_db_instance = client[mongo_db]
    return mongo_db_instance

def get_collection(mongo_db, collection_to_get):
    collection = mongo_db[collection_to_get]
    return collection

def insert_all_in_collection(list_to_add, collection):
    if len(list_to_add) > 0:
        collection.insert_many(list_to_add)

def check_article_exist(article_title):
    articles = collection_article.find({'title': article_title})
    for _ in articles:
        return True
    return False

def remove_duplicates_from_db(article_list):
    new_list = []
    for article in article_list:
        exist = check_article_exist(article['title'])
        if not exist:
            new_list.append(article)
    return new_list


In [ ]:
mongo_url = "mongodb://"
mongo_db_name = "kiev"
mongo_collection_article = "articles"
synonyms_deaths = ["death", "deaths", "murder", "killed", "kill", "died", "dying", "wounded", "wound", "missing", "casualties"]

mongo_db = connect_mongo(mongo_url, mongo_db_name)
collection_article = get_collection(mongo_db, mongo_collection_article)

articles_list = []
for i in range(1, 100):
    new_url = main_url + str(i)
    response = requests.get(new_url, headers=headers, data=payload)
    
    try:
        page_json = response.json()
    except ValueError:
        print(f"Warning: Could not parse JSON from {new_url}")
        break
    
    if page_json.get('message') == end_parsing_string:
        break

    print(f"Processing: {new_url}")
    articles_list = convert_json_page_to_articles_list(page_json, articles_list)
    
    removed_duplicate_list_db = remove_duplicates_from_db(articles_list)
    
    if len(articles_list) != len(removed_duplicate_list_db):
        print("Duplicates found in DB; stopping the fetch loop")
        articles_list = removed_duplicate_list_db
        break

    time.sleep(5)

# Remove duplicates within the new articles list itself
removed_duplicate_list = remove_duplicates(articles_list)

# Insert only the new articles not already in DB
insert_all_in_collection(removed_duplicate_list, collection_article)
